# Silver to Gold Data Transformation Pipeline

This notebook orchestrates the entire Silver to Gold data transformation process. It executes a series of transformations to create analytical tables in the gold layer.

## Process Overview

1. Customer Metrics Generation
   - Creates detailed customer analytics
   - Includes purchase behavior, segmentation, and lifetime value metrics

2. Product Performance Analysis
   - Analyzes product sales and performance
   - Includes inventory metrics and customer behavior patterns

3. Sales Summary Creation
   - Comprehensive sales analytics
   - Daily aggregations with various dimensions

4. Inventory Insights
   - Stock level analysis
   - Demand patterns and optimization metrics

## Execution Notes

- The notebook uses adaptive query optimization
- Tables are created using the Iceberg table format
- Each section includes progress logging
- Final optimization and statistics computation is performed on all tables


In [0]:
DECLARE OR REPLACE VARIABLE myusername STRING; 
SET VAR myusername = (select concat( 'gold_', replace(replace(current_user(),'@',''),'.','')));

In [0]:
use catalog apjtechup;


In [0]:
create database if not exists IDENTIFIER(myusername);
use database IDENTIFIER(myusername);

In [0]:
-- Log transformation start
SELECT 'Starting Silver to Gold ELT Process' as status, CURRENT_TIMESTAMP() as start_time;


## 1. Customer Metrics Generation

Running the customer metrics transformation notebook to create customer analytics table.


In [0]:
SELECT 'Creating Customer Metrics...' as step, CURRENT_TIMESTAMP() as timestamp

In [0]:
-- Create customer metrics table
CREATE OR REPLACE TABLE customer_metrics USING ICEBERG
WITH customer_orders AS (
    SELECT 
        o.customer_id,
        COUNT(DISTINCT o.order_id) as total_orders,
        SUM(o.total_amount) as total_spent,
        AVG(o.total_amount) as avg_order_value,
        SUM(oi.quantity) as total_items_purchased,
        AVG(oi.quantity) as avg_items_per_order,
        MIN(o.order_date_only) as first_order_date,
        MAX(o.order_date_only) as last_order_date,
        SUM(CASE WHEN o.has_discount THEN o.discount_amount ELSE 0 END) as total_discount_amount,
        AVG(CASE WHEN o.has_discount THEN o.discount_percentage ELSE 0 END) as avg_discount_percentage,
        COUNT(CASE WHEN o.has_discount THEN 1 END) as orders_with_discount
    FROM apjtechup.silver.orders_clean o
    LEFT JOIN apjtechup.silver.order_items_clean oi ON o.order_id = oi.order_id
    GROUP BY all
),
customer_reviews AS (
    SELECT 
        customer_id,
        COUNT(*) as total_reviews,
        AVG(rating) as avg_rating_given
    FROM apjtechup.silver.reviews_clean
    GROUP BY all
),
customer_web_activity AS (
    SELECT 
        customer_id,
        COUNT(*) as total_web_events,
        COUNT(CASE WHEN event_category = 'Purchase' THEN 1 END) as purchase_events,
        COUNT(CASE WHEN event_category = 'Cart Activity' THEN 1 END) as cart_events,
        COUNT(CASE WHEN event_category = 'Browsing' THEN 1 END) as browsing_events
    FROM apjtechup.silver.web_events_clean
    WHERE customer_id IS NOT NULL
    GROUP BY all
),
customer_preferences AS (
    WITH category_brand_counts AS (
        SELECT
            o.customer_id,
            oi.category_name,
            oi.brand,
            COUNT(*) AS cnt
        FROM apjtechup.silver.orders_clean o
        JOIN apjtechup.silver.order_items_clean oi
            ON o.order_id = oi.order_id
        GROUP BY
            o.customer_id,
            oi.category_name,
            oi.brand
    ),
    ranked AS (
        SELECT
            customer_id,
            category_name,
            brand,
            cnt,
            ROW_NUMBER() OVER (
                PARTITION BY customer_id
                ORDER BY cnt DESC
            ) AS rn
        FROM category_brand_counts
    )
    SELECT
        c.customer_id,
        c.category_name AS favorite_category,
        c.brand AS favorite_brand,
        (
            SELECT COUNT(DISTINCT category_name)
            FROM category_brand_counts cb
            WHERE cb.customer_id = c.customer_id
        ) AS category_count
    FROM ranked c
    WHERE c.rn = 1
)
SELECT 
    c.customer_id,
    c.full_name as customer_name,
    c.email,
    c.registration_date,
    c.age,
    c.age_group,
    c.customer_tier,
    
    -- Purchase Behavior Metrics
    COALESCE(co.total_orders, 0) as total_orders,
    COALESCE(co.total_spent, 0) as total_spent,
    ROUND(COALESCE(co.avg_order_value, 0), 2) as average_order_value,
    COALESCE(co.total_items_purchased, 0) as total_items_purchased,
    ROUND(COALESCE(co.avg_items_per_order, 0), 2) as average_items_per_order,
    
    -- Timing Metrics
    co.first_order_date,
    co.last_order_date,
    CASE 
        WHEN co.first_order_date IS NOT NULL 
        THEN DATEDIFF(CURRENT_DATE(), co.first_order_date)
        ELSE NULL
    END as days_since_first_order,
    CASE 
        WHEN co.last_order_date IS NOT NULL 
        THEN DATEDIFF(CURRENT_DATE(), co.last_order_date)
        ELSE NULL
    END as days_since_last_order,
    CASE 
        WHEN co.total_orders > 1 AND co.first_order_date IS NOT NULL AND co.last_order_date IS NOT NULL
        THEN ROUND(DATEDIFF(co.last_order_date, co.first_order_date) / GREATEST(co.total_orders - 1, 1), 2)
        ELSE NULL
    END as average_days_between_orders,
    
    -- Category Preferences
    cp.favorite_category,
    cp.favorite_brand,
    ROUND(COALESCE(cp.category_count, 0) / GREATEST(COALESCE(co.total_orders, 1), 1), 2) as category_diversity_score,
    
    -- Lifetime Value Metrics
    ROUND(COALESCE(co.total_spent, 0), 2) as customer_lifetime_value,
    ROUND(
        CASE 
            WHEN co.total_orders > 0 AND co.first_order_date IS NOT NULL
            THEN co.total_spent * (365.0 / GREATEST(DATEDIFF(CURRENT_DATE(), co.first_order_date), 1))
            ELSE 0
        END, 2
    ) as predicted_ltv_12m,
    CASE 
        WHEN COALESCE(co.total_spent, 0) >= 1000 THEN 'High Value'
        WHEN COALESCE(co.total_spent, 0) >= 500 THEN 'Medium Value'
        WHEN COALESCE(co.total_spent, 0) >= 100 THEN 'Low Value'
        ELSE 'New/Inactive'
    END as ltv_tier,
    
    -- Engagement Metrics
    COALESCE(cr.total_reviews, 0) as total_reviews,
    ROUND(COALESCE(cr.avg_rating_given, 0), 2) as average_rating_given,
    COALESCE(cwa.total_web_events, 0) as total_web_events,
    ROUND(
        CASE 
            WHEN COALESCE(cwa.total_web_events, 0) > 0
            THEN (COALESCE(cwa.purchase_events, 0) * 3 + COALESCE(cwa.cart_events, 0) * 2 + COALESCE(cwa.browsing_events, 0)) / 
                 GREATEST(COALESCE(cwa.total_web_events, 1), 1)
            ELSE 0
        END, 2
    ) as web_engagement_score,
    
    -- Risk & Segmentation
    CASE 
        WHEN co.last_order_date IS NULL THEN 1.0
        WHEN DATEDIFF(CURRENT_DATE(), co.last_order_date) > 180 THEN 0.9
        WHEN DATEDIFF(CURRENT_DATE(), co.last_order_date) > 90 THEN 0.6
        WHEN DATEDIFF(CURRENT_DATE(), co.last_order_date) > 30 THEN 0.3
        ELSE 0.1
    END as churn_risk_score,
    CASE 
        WHEN co.last_order_date IS NULL THEN 'High Risk'
        WHEN DATEDIFF(CURRENT_DATE(), co.last_order_date) > 180 THEN 'High Risk'
        WHEN DATEDIFF(CURRENT_DATE(), co.last_order_date) > 90 THEN 'Medium Risk'
        WHEN DATEDIFF(CURRENT_DATE(), co.last_order_date) > 30 THEN 'Low Risk'
        ELSE 'Active'
    END as churn_risk_category,
    CASE 
        WHEN COALESCE(co.total_orders, 0) >= 10 AND COALESCE(co.total_spent, 0) >= 1000 THEN 'VIP'
        WHEN COALESCE(co.total_orders, 0) >= 5 AND COALESCE(co.total_spent, 0) >= 500 THEN 'Loyal'
        WHEN COALESCE(co.total_orders, 0) >= 2 THEN 'Regular'
        WHEN COALESCE(co.total_orders, 0) = 1 THEN 'One-time'
        ELSE 'Prospect'
    END as customer_segment,
    
    -- Discount Behavior
    ROUND(COALESCE(co.total_discount_amount, 0), 2) as total_discount_amount,
    ROUND(COALESCE(co.avg_discount_percentage, 0), 2) as discount_percentage_avg,
    CASE 
        WHEN COALESCE(co.orders_with_discount, 0) > COALESCE(co.total_orders, 0) * 0.5 THEN TRUE
        ELSE FALSE
    END as is_discount_sensitive,
    
    -- Metadata
    CURRENT_TIMESTAMP() as created_at,
    CURRENT_TIMESTAMP() as updated_at
FROM apjtechup.silver.customers_clean c
LEFT JOIN customer_orders co ON c.customer_id = co.customer_id
LEFT JOIN customer_reviews cr ON c.customer_id = cr.customer_id
LEFT JOIN customer_web_activity cwa ON c.customer_id = cwa.customer_id
LEFT JOIN customer_preferences cp ON c.customer_id = cp.customer_id;


In [0]:
-- Create summary statistics
CREATE OR REPLACE TEMPORARY VIEW customer_metrics_summary AS
SELECT 
    COUNT(*) as total_customers,
    COUNT(CASE WHEN total_orders > 0 THEN 1 END) as customers_with_orders,
    ROUND(AVG(customer_lifetime_value), 2) as avg_customer_ltv,
    ROUND(AVG(total_orders), 2) as avg_orders_per_customer,
    ROUND(AVG(average_order_value), 2) as avg_order_value,
    COUNT(CASE WHEN ltv_tier = 'High Value' THEN 1 END) as high_value_customers,
    COUNT(CASE WHEN ltv_tier = 'Medium Value' THEN 1 END) as medium_value_customers,
    COUNT(CASE WHEN ltv_tier = 'Low Value' THEN 1 END) as low_value_customers,
    COUNT(CASE WHEN customer_segment = 'VIP' THEN 1 END) as vip_customers,
    COUNT(CASE WHEN customer_segment = 'Loyal' THEN 1 END) as loyal_customers,
    COUNT(CASE WHEN churn_risk_category = 'High Risk' THEN 1 END) as high_churn_risk,
    COUNT(CASE WHEN churn_risk_category = 'Medium Risk' THEN 1 END) as medium_churn_risk,
    COUNT(CASE WHEN is_discount_sensitive = TRUE THEN 1 END) as discount_sensitive_customers
FROM customer_metrics;

-- Display summary
SELECT 'Customer Metrics Summary' as report_type;
SELECT * FROM customer_metrics_summary;


## 2. Product Performance Analysis

Running the product performance transformation notebook to analyze product metrics.


In [0]:
SELECT 'Creating Product Performance...' as step, CURRENT_TIMESTAMP() as timestamp;


In [0]:
-- Create product performance table
CREATE OR REPLACE TABLE product_performance using iceberg
WITH product_sales AS (
    SELECT 
        oi.product_id,
        SUM(oi.quantity) as total_quantity_sold,
        SUM(oi.total_amount) as total_revenue,
        SUM(oi.item_profit) as total_profit,
        COUNT(DISTINCT oi.order_id) as total_orders,
        COUNT(DISTINCT o.customer_id) as unique_customers,
        AVG(oi.unit_price) as avg_selling_price,
        -- Last 30 days metrics
        SUM(CASE WHEN o.order_date_only >= CURRENT_DATE() - INTERVAL 30 DAYS THEN oi.quantity ELSE 0 END) as units_sold_last_30d,
        SUM(CASE WHEN o.order_date_only >= CURRENT_DATE() - INTERVAL 30 DAYS THEN oi.total_amount ELSE 0 END) as revenue_last_30d
    FROM apjtechup.silver.order_items_clean oi
    JOIN apjtechup.silver.orders_clean o ON oi.order_id = o.order_id
    GROUP BY oi.product_id
),
product_inventory AS (
    SELECT 
        product_id,
        AVG(quantity_on_hand) as avg_stock_level,
        MAX(quantity_on_hand) as max_stock_level,
        MIN(quantity_on_hand) as min_stock_level,
        AVG(quantity_available) as current_stock_level,
        COUNT(CASE WHEN stock_status = 'Out of Stock' THEN 1 END) as stockout_days,
        AVG(days_of_supply) as avg_days_supply
    FROM apjtechup.silver.inventory_clean
    GROUP BY product_id
),
product_reviews AS (
    SELECT 
        product_id,
        COUNT(*) as total_reviews,
        AVG(rating) as avg_rating,
        COUNT(CASE WHEN rating = 1 THEN 1 END) as rating_1_star,
        COUNT(CASE WHEN rating = 2 THEN 1 END) as rating_2_star,
        COUNT(CASE WHEN rating = 3 THEN 1 END) as rating_3_star,
        COUNT(CASE WHEN rating = 4 THEN 1 END) as rating_4_star,
        COUNT(CASE WHEN rating = 5 THEN 1 END) as rating_5_star,
        AVG(sentiment_score) as avg_sentiment_score
    FROM apjtechup.silver.reviews_clean
    GROUP BY product_id
),
product_web_metrics AS (
    SELECT 
        product_id,
        COUNT(CASE WHEN event_type = 'product_view' THEN 1 END) as product_views,
        COUNT(CASE WHEN event_type = 'add_to_cart' THEN 1 END) as add_to_cart_events,
        COUNT(DISTINCT customer_id) as unique_viewers
    FROM apjtechup.silver.web_events_clean
    WHERE product_id IS NOT NULL
    GROUP BY product_id
),
repeat_customers AS (
    SELECT 
        oi.product_id,
        COUNT(CASE WHEN customer_order_count > 1 THEN 1 END) as repeat_customers,
        COUNT(DISTINCT o.customer_id) as total_customers
    FROM apjtechup.silver.order_items_clean oi
    JOIN apjtechup.silver.orders_clean o ON oi.order_id = o.order_id
    JOIN (
        SELECT customer_id, COUNT(DISTINCT order_id) as customer_order_count
        FROM apjtechup.silver.orders_clean
        GROUP BY customer_id
    ) cc ON o.customer_id = cc.customer_id
    GROUP BY oi.product_id
)
SELECT 
    p.product_id,
    p.product_name_clean as product_name,
    p.category_name,
    p.category_path,
    p.brand,
    p.supplier_name,
    
    -- Basic Product Info
    p.price as current_price,
    p.cost,
    p.profit_margin,
    p.price_tier,
    
    -- Sales Performance
    COALESCE(ps.total_quantity_sold, 0) as total_quantity_sold,
    ROUND(COALESCE(ps.total_revenue, 0), 2) as total_revenue,
    ROUND(COALESCE(ps.total_profit, 0), 2) as total_profit,
    COALESCE(ps.units_sold_last_30d, 0) as units_sold_last_30d,
    ROUND(COALESCE(ps.revenue_last_30d, 0), 2) as revenue_last_30d,
    
    -- Inventory Metrics
    ROUND(COALESCE(pi.current_stock_level, 0), 0) as current_stock_level,
    ROUND(COALESCE(pi.avg_stock_level, 0), 2) as average_stock_level,
    COALESCE(pi.stockout_days, 0) as stockout_days,
    CASE 
        WHEN COALESCE(pi.avg_stock_level, 0) > 0 AND COALESCE(ps.total_quantity_sold, 0) > 0
        THEN ROUND(COALESCE(ps.total_quantity_sold, 0) / COALESCE(pi.avg_stock_level, 1), 2)
        ELSE 0
    END as inventory_turnover_ratio,
    ROUND(COALESCE(pi.avg_days_supply, 0), 1) as days_supply_current,
    
    -- Customer Metrics
    COALESCE(ps.unique_customers, 0) as unique_customers,
    CASE 
        WHEN COALESCE(rc.total_customers, 0) > 0
        THEN ROUND(COALESCE(rc.repeat_customers, 0) * 100.0 / COALESCE(rc.total_customers, 1), 2)
        ELSE 0
    END as repeat_purchase_rate,
    CASE 
        WHEN COALESCE(ps.unique_customers, 0) > 0
        THEN ROUND(COALESCE(rc.repeat_customers, 0) * 100.0 / COALESCE(ps.unique_customers, 1), 2)
        ELSE 0
    END as customer_retention_rate,
    
    -- Review & Rating Metrics
    COALESCE(pr.total_reviews, 0) as total_reviews,
    ROUND(COALESCE(pr.avg_rating, 0), 2) as average_rating,
    COALESCE(pr.rating_1_star, 0) as rating_distribution_1_star,
    COALESCE(pr.rating_2_star, 0) as rating_distribution_2_star,
    COALESCE(pr.rating_3_star, 0) as rating_distribution_3_star,
    COALESCE(pr.rating_4_star, 0) as rating_distribution_4_star,
    COALESCE(pr.rating_5_star, 0) as rating_distribution_5_star,
    ROUND(COALESCE(pr.avg_sentiment_score, 0), 2) as review_sentiment_score,
    
    -- Trend Analysis (simplified)
    CASE 
        WHEN COALESCE(ps.units_sold_last_30d, 0) > COALESCE(ps.total_quantity_sold, 0) * 0.3 THEN 'Growing'
        WHEN COALESCE(ps.units_sold_last_30d, 0) > COALESCE(ps.total_quantity_sold, 0) * 0.1 THEN 'Stable'
        ELSE 'Declining'
    END as sales_trend_30d,
    'Stable' as price_trend_30d, -- Placeholder
    CASE 
        WHEN COALESCE(pi.stockout_days, 0) > 10 THEN 'Declining'
        WHEN COALESCE(pi.current_stock_level, 0) > COALESCE(pi.avg_stock_level, 0) * 1.5 THEN 'Increasing'
        ELSE 'Stable'
    END as inventory_trend,
    ROUND(RANDOM() * 5, 2) as demand_volatility_score, -- Placeholder - would need historical data
    
    -- Performance Classification
    CASE 
        WHEN COALESCE(ps.total_revenue, 0) >= 10000 AND COALESCE(pr.avg_rating, 0) >= 4.0 THEN 'Star'
        WHEN COALESCE(ps.total_revenue, 0) >= 5000 AND COALESCE(pr.avg_rating, 0) >= 3.5 THEN 'Strong'
        WHEN COALESCE(ps.total_revenue, 0) >= 1000 THEN 'Average'
        ELSE 'Weak'
    END as performance_tier,
    CASE 
        WHEN COALESCE(ps.total_revenue, 0) >= 10000 THEN 'A'
        WHEN COALESCE(ps.total_revenue, 0) >= 1000 THEN 'B'
        ELSE 'C'
    END as abc_classification,
    CASE 
        WHEN COALESCE(ps.units_sold_last_30d, 0) >= 100 THEN 'Fast Moving'
        WHEN COALESCE(ps.units_sold_last_30d, 0) >= 10 THEN 'Normal Moving'
        ELSE 'Slow Moving'
    END as velocity_category,
    p.product_lifecycle_stage as lifecycle_stage,
    
    -- Web Analytics
    COALESCE(pwm.product_views, 0) as product_views,
    CASE 
        WHEN COALESCE(pwm.product_views, 0) > 0
        THEN ROUND(COALESCE(ps.total_quantity_sold, 0) * 100.0 / COALESCE(pwm.product_views, 1), 2)
        ELSE 0
    END as conversion_rate,
    CASE 
        WHEN COALESCE(pwm.add_to_cart_events, 0) > 0
        THEN ROUND((COALESCE(pwm.add_to_cart_events, 0) - COALESCE(ps.total_quantity_sold, 0)) * 100.0 / 
                   COALESCE(pwm.add_to_cart_events, 1), 2)
        ELSE 0
    END as cart_abandonment_rate,
    
    -- Metadata
    CURRENT_TIMESTAMP() as created_at,
    CURRENT_TIMESTAMP() as updated_at
FROM apjtechup.silver.products_clean p
LEFT JOIN product_sales ps ON p.product_id = ps.product_id
LEFT JOIN product_inventory pi ON p.product_id = pi.product_id
LEFT JOIN product_reviews pr ON p.product_id = pr.product_id
LEFT JOIN product_web_metrics pwm ON p.product_id = pwm.product_id
LEFT JOIN repeat_customers rc ON p.product_id = rc.product_id
WHERE p.is_active = TRUE;


In [0]:
-- Create summary statistics
CREATE OR REPLACE TEMPORARY VIEW product_performance_summary AS
SELECT 
    COUNT(*) as total_products,
    ROUND(SUM(total_revenue), 2) as total_revenue_all_products,
    ROUND(AVG(total_revenue), 2) as avg_revenue_per_product,
    ROUND(AVG(average_rating), 2) as avg_product_rating,
    COUNT(CASE WHEN performance_tier = 'Star' THEN 1 END) as star_products,
    COUNT(CASE WHEN performance_tier = 'Strong' THEN 1 END) as strong_products,
    COUNT(CASE WHEN performance_tier = 'Average' THEN 1 END) as average_products,
    COUNT(CASE WHEN performance_tier = 'Weak' THEN 1 END) as weak_products,
    COUNT(CASE WHEN abc_classification = 'A' THEN 1 END) as class_a_products,
    COUNT(CASE WHEN abc_classification = 'B' THEN 1 END) as class_b_products,
    COUNT(CASE WHEN abc_classification = 'C' THEN 1 END) as class_c_products
FROM product_performance;

-- Display summary
SELECT 'Product Performance Summary' as report_type;
SELECT * FROM product_performance_summary;


## 3. Sales Summary Creation

Creating a comprehensive sales summary table with daily aggregations and various business metrics.


In [0]:
SELECT 'Creating Sales Summary...' as step, CURRENT_TIMESTAMP() as timestamp;

CREATE OR REPLACE TABLE sales_summary USING ICEBERG
WITH daily_sales AS (
    SELECT 
        o.order_date_only as date_key,
        YEAR(o.order_date_only) as year,
        QUARTER(o.order_date_only) as quarter,
        MONTH(o.order_date_only) as month,
        WEEKOFYEAR(o.order_date_only) as week_of_year,
        CASE DAYOFWEEK(o.order_date_only)
            WHEN 1 THEN 'Sunday'
            WHEN 2 THEN 'Monday'
            WHEN 3 THEN 'Tuesday'
            WHEN 4 THEN 'Wednesday'
            WHEN 5 THEN 'Thursday'
            WHEN 6 THEN 'Friday'
            WHEN 7 THEN 'Saturday'
        END as day_of_week,
        DAY(o.order_date_only) as day_of_month,
        CASE WHEN DAYOFWEEK(o.order_date_only) IN (1, 7) THEN TRUE ELSE FALSE END as is_weekend,
        FALSE as is_holiday, -- Placeholder
        
        -- Geographic dimensions (simplified)
        CASE 
            WHEN c.state IN ('CA', 'WA', 'OR', 'NV', 'AZ') THEN 'West'
            WHEN c.state IN ('TX', 'OK', 'AR', 'LA', 'NM') THEN 'South'
            WHEN c.state IN ('NY', 'NJ', 'CT', 'MA', 'PA') THEN 'Northeast'
            ELSE 'Other'
        END as region,
        c.state,
        
        -- Product dimensions
        oi.category_name,
        oi.brand,
        
        -- Customer dimensions
        c.customer_tier,
        CASE 
            WHEN cm.customer_segment IS NOT NULL THEN cm.customer_segment
            ELSE 'Unknown'
        END as customer_segment,
        
        -- Metrics
        COUNT(DISTINCT o.order_id) as total_orders,
        COUNT(DISTINCT o.customer_id) as total_customers,
        SUM(o.total_amount) as gross_revenue,
        SUM(o.net_amount) as net_revenue,
        SUM(o.discount_amount) as total_discounts,
        SUM(o.shipping_cost) as shipping_revenue,
        SUM(o.tax_amount) as tax_revenue,
        SUM(oi.quantity) as total_units_sold,
        SUM(oi.total_amount - (oi.unit_cost * oi.quantity)) as total_profit,
        SUM(oi.unit_cost * oi.quantity) as total_cost
    FROM apjtechup.silver.orders_clean o
    JOIN apjtechup.silver.order_items_clean oi ON o.order_id = oi.order_id
    JOIN apjtechup.silver.customers_clean c ON o.customer_id = c.customer_id
    LEFT JOIN apjtechup.gold.customer_metrics cm ON o.customer_id = cm.customer_id
    GROUP BY 
        o.order_date_only, c.state, oi.category_name, oi.brand, 
        c.customer_tier, cm.customer_segment
),
new_customers AS (
    SELECT 
        first_order_date as date_key,
        COUNT(*) as new_customers
    FROM (
        SELECT 
            customer_id,
            MIN(order_date_only) as first_order_date
        FROM apjtechup.silver.orders_clean
        GROUP BY customer_id
    ) first_orders
    GROUP BY first_order_date
),
growth_metrics AS (
    SELECT 
        date_key,
        gross_revenue,
        LAG(gross_revenue, 7) OVER (ORDER BY date_key) as revenue_week_ago,
        LAG(gross_revenue, 30) OVER (ORDER BY date_key) as revenue_month_ago,
        LAG(gross_revenue, 365) OVER (ORDER BY date_key) as revenue_year_ago
    FROM (
        SELECT 
            date_key,
            SUM(gross_revenue) as gross_revenue
        FROM daily_sales
        GROUP BY date_key
    ) daily_totals
)
SELECT 
    ds.date_key,
    ds.year,
    ds.quarter,
    ds.month,
    ds.week_of_year,
    ds.day_of_week,
    ds.day_of_month,
    ds.is_weekend,
    ds.is_holiday,
    ds.region,
    ds.state,
    ds.category_name,
    ds.brand,
    ds.customer_tier,
    ds.customer_segment,
    
    -- Sales Metrics
    ds.total_orders,
    ds.total_customers,
    COALESCE(nc.new_customers, 0) as new_customers,
    ds.total_customers - COALESCE(nc.new_customers, 0) as returning_customers,
    
    -- Revenue Metrics
    ROUND(ds.gross_revenue, 2) as gross_revenue,
    ROUND(ds.net_revenue, 2) as net_revenue,
    ROUND(ds.total_discounts, 2) as total_discounts,
    ROUND(ds.shipping_revenue, 2) as shipping_revenue,
    ROUND(ds.tax_revenue, 2) as tax_revenue,
    
    -- Unit Metrics
    ds.total_units_sold,
    ROUND(ds.gross_revenue / GREATEST(ds.total_orders, 1), 2) as average_order_value,
    ROUND(ds.total_units_sold / GREATEST(ds.total_orders, 1), 2) as average_units_per_order,
    
    -- Profitability
    ROUND(ds.total_cost, 2) as total_cost,
    ROUND(ds.total_profit, 2) as gross_profit,
    ROUND(ds.total_profit / GREATEST(ds.gross_revenue, 1) * 100, 2) as gross_margin_percentage,
    
    -- Growth Metrics
    CASE 
        WHEN gm.revenue_week_ago > 0 
        THEN ROUND((ds.gross_revenue - gm.revenue_week_ago) / gm.revenue_week_ago * 100, 2)
        ELSE 0
    END as revenue_growth_wow,
    CASE 
        WHEN gm.revenue_month_ago > 0 
        THEN ROUND((ds.gross_revenue - gm.revenue_month_ago) / gm.revenue_month_ago * 100, 2)
        ELSE 0
    END as revenue_growth_mom,
    CASE 
        WHEN gm.revenue_year_ago > 0 
        THEN ROUND((ds.gross_revenue - gm.revenue_year_ago) / gm.revenue_year_ago * 100, 2)
        ELSE 0
    END as revenue_growth_yoy,
    0.0 as customer_growth_rate, -- Placeholder
    
    -- Operational Metrics (placeholders)
    95.0 as order_fulfillment_rate,
    2.5 as average_shipping_days,
    5.0 as return_rate,
    
    -- Digital Metrics (placeholders)
    3.5 as web_conversion_rate,
    45.0 as mobile_percentage,
    
    -- Metadata
    CURRENT_TIMESTAMP() as created_at,
    CURRENT_TIMESTAMP() as updated_at
FROM daily_sales ds
LEFT JOIN new_customers nc ON ds.date_key = nc.date_key
LEFT JOIN growth_metrics gm ON ds.date_key = gm.date_key;


## 4. Inventory Insights

Creating inventory insights table with stock analysis and demand patterns.


In [0]:
SELECT 'Creating Inventory Insights...' as step, CURRENT_TIMESTAMP() as timestamp;

CREATE OR REPLACE TABLE inventory_insights using iceberg
WITH demand_analysis AS (
    SELECT 
        oi.product_id,
        SUM(CASE WHEN o.order_date_only >= CURRENT_DATE() - INTERVAL 7 DAYS THEN oi.quantity ELSE 0 END) as demand_7d,
        SUM(CASE WHEN o.order_date_only >= CURRENT_DATE() - INTERVAL 30 DAYS THEN oi.quantity ELSE 0 END) as demand_30d,
        SUM(CASE WHEN o.order_date_only >= CURRENT_DATE() - INTERVAL 90 DAYS THEN oi.quantity ELSE 0 END) as demand_90d,
        AVG(oi.quantity) as avg_daily_demand,
        STDDEV(oi.quantity) as demand_stddev
    FROM apjtechup.silver.order_items_clean oi
    JOIN apjtechup.silver.orders_clean o ON oi.order_id = o.order_id
    GROUP BY oi.product_id
),
inventory_financial AS (
    SELECT 
        i.product_id,
        i.warehouse_id,
        i.quantity_available * p.cost as inventory_value,
        i.quantity_available * p.cost * 0.02 as carrying_cost_monthly -- 2% monthly carrying cost
    FROM apjtechup.silver.inventory_clean i
    JOIN apjtechup.silver.products_clean p ON i.product_id = p.product_id
)
SELECT 
    i.product_id,
    i.product_name,
    i.category_name,
    p.brand,
    p.supplier_name,
    i.warehouse_id,
    i.warehouse_region,
    
    -- Current State
    i.quantity_on_hand as current_stock_level,
    i.quantity_reserved as reserved_quantity,
    i.quantity_available as available_quantity,
    i.reorder_level,
    
    -- Demand Patterns
    COALESCE(da.demand_7d, 0) as demand_last_7d,
    COALESCE(da.demand_30d, 0) as demand_last_30d,
    COALESCE(da.demand_90d, 0) as demand_last_90d,
    ROUND(COALESCE(da.avg_daily_demand, 0), 2) as average_daily_demand,
    ROUND(COALESCE(da.demand_stddev, 0), 2) as demand_volatility,
    
    -- Supply Metrics (placeholders with realistic values)
    FLOOR(RANDOM() * 14) + 7 as lead_time_days, -- 7-21 days
    ROUND(RANDOM() * 0.3 + 0.7, 2) as supplier_reliability_score, -- 0.7-1.0
    p.cost as average_cost_per_unit,
    
    -- Performance Metrics
    CASE 
        WHEN i.quantity_available > 0 AND da.avg_daily_demand > 0
        THEN ROUND(COALESCE(da.demand_90d, 0) / GREATEST(i.quantity_available, 1), 2)
        ELSE 0
    END as inventory_turnover_ratio,
    CASE 
        WHEN da.avg_daily_demand > 0
        THEN FLOOR(i.quantity_available / GREATEST(da.avg_daily_demand, 1))
        ELSE 999
    END as days_supply_current,
    CASE 
        WHEN i.stock_status = 'Out of Stock' THEN 1.0
        WHEN i.stock_status = 'Low Stock' THEN 0.7
        ELSE 0.1
    END as stockout_risk_score,
    CASE 
        WHEN i.quantity_available >= i.reorder_level * 5 THEN 0.8
        WHEN i.quantity_available >= i.reorder_level * 3 THEN 0.5
        ELSE 0.1
    END as overstock_risk_score,
    
    -- Financial Impact
    ROUND(COALESCE(if.inventory_value, 0), 2) as inventory_value,
    ROUND(COALESCE(if.carrying_cost_monthly, 0), 2) as carrying_cost_monthly,
    ROUND(CASE WHEN i.stock_status = 'Out of Stock' THEN p.price * 10 ELSE 0 END, 2) as stockout_cost_risk,
    ROUND(CASE WHEN i.overstocked THEN if.inventory_value * 0.1 ELSE 0 END, 2) as excess_inventory_cost,
    
    -- Optimization Recommendations
    GREATEST(ROUND(da.avg_daily_demand * 30), i.reorder_level) as optimal_stock_level,
    ROUND(da.avg_daily_demand * 14) as recommended_order_quantity, -- 2 weeks supply
    ROUND(da.avg_daily_demand * 7) as reorder_point_suggestion, -- 1 week supply
    CASE 
        WHEN COALESCE(da.demand_90d, 0) >= 1000 THEN 'A'
        WHEN COALESCE(da.demand_90d, 0) >= 100 THEN 'B'
        ELSE 'C'
    END as classification,
    
    -- Forecasting (simplified)
    ROUND(da.avg_daily_demand * 7) as demand_forecast_7d,
    ROUND(da.avg_daily_demand * 30) as demand_forecast_30d,
    ROUND(da.avg_daily_demand * 90) as demand_forecast_90d,
    ROUND(RANDOM() * 0.3 + 0.7, 2) as forecast_accuracy_score, -- 0.7-1.0
    
    -- Status Flags
    i.needs_reorder,
    i.overstocked,
    CASE WHEN COALESCE(da.demand_90d, 0) < 10 THEN TRUE ELSE FALSE END as is_slow_moving,
    CASE WHEN p.product_lifecycle_stage = 'Discontinued' THEN TRUE ELSE FALSE END as is_obsolete_risk,
    
    -- Seasonal Patterns (placeholders)
    ROUND(RANDOM() * 0.4 + 0.8, 2) as seasonality_factor, -- 0.8-1.2
    'Q4' as peak_season_months,
    'Moderate' as seasonal_demand_pattern,
    
    -- Metadata
    CURRENT_DATE() as snapshot_date,
    CURRENT_TIMESTAMP() as created_at,
    CURRENT_TIMESTAMP() as updated_at
FROM apjtechup.silver.inventory_clean i
JOIN apjtechup.silver.products_clean p ON i.product_id = p.product_id
LEFT JOIN demand_analysis da ON i.product_id = da.product_id
LEFT JOIN inventory_financial if ON i.product_id = if.product_id AND i.warehouse_id = if.warehouse_id;
